In [1]:
from icecream import ic, install
install()
ic.disable()

In [2]:
from spacy import displacy
from spacy import blank
from spacy import load

from MyUtilty.tag2pos import tag2posFactory
from MyUtilty.psoPipe import PendingAspectsOpinionsPipe
from MyUtilty.cost_seg import CostSegmenter
from MyUtilty.emoBankSearch import EmoBankSearch

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-01 09:56:35.809879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-01 09:56:35.827291: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-01 09:56:35.832620: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-01 09:56:35.846327: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [3]:
from spacy.tokens import Doc
Doc.set_extension("pending_aspects_opinions_i", default=[])
Doc.set_extension("pending_aspects_opinions_text", default=[])
Doc.set_extension("aspect_sentiment_triplets", default=[])

In [4]:
text = ['''曾經我差點也被某個朋友拉去上課， 也是講一些不去上課就不會改變人生不會成功的話， 還好身邊還有幾個清醒的朋友， 直接戳破這些話術， 說如果真的想改變， 書店就很多書了， 網路上也有很多免費資源能看， 真的沒必要花這麼貴的錢去上課。 加上自己那時候剛畢業還沒找到工作也真的沒錢， 實在沒辦法做出那個拉人的朋友所說， 去借錢貸款上課的行為， 後來我就明確告訴那個朋友我的想法， 講完他還想拉我去上課， 我就不理他了。 這篇文章是我當初爬文超久才找到的， 一開始是看痞課邦有人轉發， 雖然文章年代久遠， 名稱會換來會去， 但本質上是差不多的， 換湯不換藥， 其他機構的課程也大同小異。 這篇文章皆露的內容， 對我幫助滿大的， 真心佩服及感謝寫這篇文章的記者， 我已經找到原文的完整內容了， 上面金額都是人民幣， 我有參考痞課邦寫的金額寫在旁邊簡單換算， 上面的課程金額現在至少要再翻倍， 希望能讓這邊的大家也有幫助。 ------------------------------------------------------------ 懶得看字的 建議先看這個記者於2021的專訪影片 裡面就有稍微提到2007年她當臥底的事--------------‐--------- 原文記者李亞玲 2007年於 成都商报 擔任記者 臥底半年所寫 【揭秘菁英課程生命泉源：強迫洗腦的過程】 訪談手記 好奇害死貓 第一次知道「生命泉源」是在2003年7月初。 一王姓讀者打電話給我， 盛情邀請我去參加他和他的朋友們的一次社會活動 ———選一處貧困鄉村建一所希望小學。 希望小學的校址最後選定在蒲江縣西崍鎮鐵牛村。 2003年7月19日上午8時30分， 我們在成雅高速蒲江站出口處集合， 相約前往鐵牛村商討具體建校方案。 很快就來了一個由別克、豐田、尼桑、三菱等9輛豪華轎車組成的車隊。 這些神祕人士都是三、四十歲的中年人， 有的還特地帶了未成年的兒女到鐵牛村去「體驗」生活。 這些人在成都一家公司參加了一個「很有價值的課程」。 所有學員所繳的學費中的一部分會劃入公司內部的一個慈善基金， 用於捐建希望小學。 我很好奇，可一打聽， 這個課程的學費很昂貴， 讀完三個階段(一個階段的上課時間僅幾天)需要花2萬元(約台幣10萬)左右， 也就打消了念頭。 但他們都鄭重其事地說： “學費聽起來貴，可是和你學到的東西相比， 一點也不貴，很值！” 一晃3年多過去了。 2006年11月，因為做一組「富人暗中做慈善」的調查報道， 我再次聯絡他們了。 久別重逢，大家興致很高，談得很投機。 他們再次提到了那個神秘課程。 這幾年來，課程在成都發展迅速， 現在每個月的訓練課程都會爆滿。 Z先生，一大型化工企業的法人代表； X女士，成都某實權部門的前任局長。 他們都盛讚課程的功效。 Z說：「學了這個課程，我把公司做大了一倍。」 X則說：「我上了課以後，才弄清楚我到底想要一種什麼樣的生活，所以我辭職了，現在在做這個課程的授課導師。 這究竟是一門怎樣的神奇課程？ 為什麼可以為不同的人帶來不同的神奇改變？ 我的好奇心再次被勾起。 但他們都不肯透露有關課程的具體內容， 只說：「那是一個體驗式的課程，沒有教材，也不用做筆記。 導師會透過一些遊戲和練習來讓大家體驗、感悟。 它不會具體教你什麼， 它只是給你一個工具， 但有了它，你可以運用於生活中的每件事，事半功倍。 此後的兩個月裡，我們多次在茶樓聚會。 每次聚會，他們都會不約而同地談到那個課程， 不知不覺地運用一些“課程語言”， 例如“感召”、“體驗”、“轉移、”“活在當下”等。 他們透露的支言片語，讓我的好奇心不斷累積。 今年1月，我決定去上這個課程。 職業敏感告訴我， 這是一個新生事物， 它進入了我們社會的精英階層， 並且正發揮著越來越大的影響力， 我必須先去關注它、了解它。 我就讀的公司將課程分為「探索」、「突破」、「領導鍛鍊」三個階段。 三個階段每月都只開設1班。 「探索」為3個晚上增加2個全天近40個小時，學費3300元(約台幣18500元)； 「突破」為五個白天4個晚上，學費為6900元(約台幣36000元)； 「領導鍛鍊」為3個月，每個月公司會安排1個週末上課，平時每個週末則由小組長們(義務回公司服務的老學員)組織開會，學費為3900元(約台幣19000元)。 要學完三個階段， 需要5個月左右。 6月3日晚，我終於「畢業」了， 成為了一個「LP」(課程術語，指讀完三個階段的學員)。 這次親身體驗對我來說可謂驚心動魄。 並不是因為遭受了什麼來自外界的威脅， 而是這個課程在我的思想領域掀起了一番驚濤駭浪。 我對課程的心態經歷了一系列複雜的轉變： 好奇、感動、震動、糊塗、懷疑、批判… 走出課堂之後， 我又進行了半個多月的外圍訪談和深入調查。 根據掌握的資料，我最終認為， 這個課程的設計含有「心理控制」成份， 並不適合一般人。 它將心理手段運用於商業活動， 有重大隱患：它「摧毀」學員原有的心理結構， 卻未設計「協助重建心理結構」程序。 除非學員本身已具有堅定的信念、成熟的思辨能力， 否則可能會遭受心理傷害。 而這個傷害可能是致命的 ──國內外發生了多起學員上課後精神失常乃至自殺身亡的案例！ 在此，我們鄭重給讀者們一個警示： 慎上這種所謂的「菁英課程」！ 二、親身體驗 過去幾年，這門課程在成都的發展速度驚人。 到如今，每月被親友同事「感召」進課堂的學員已達到了數百人。 根據粗略估算，這門課程每年在成都收取的學費數以千萬計。 為了對這越來越具社會影響力的新生事物做出盡可能深入、客觀、全面地解讀， 今年1至6月， 記者在一位元老級「LP」(課程術語，指完成了三階段培訓的學員)的「感召」下， 先後花費2萬元， 進入經營課程的「成都XX資訊顧問有限公司」全程體驗了5個月， 揭開了這門天價「體驗式」課程的神秘面紗。 第一階 你是否一直在玩同樣的遊戲 當教室中開始傳出抽泣聲時， 音樂聲會更悲壯激昂。 導師也會完沉浸在自己的講敘中，聲淚俱下。 記者曾悄悄睜眼，只見滿室幽光， 部分男女學員已情緒失控，搥胸頓足、嚎啕大哭、呼天搶地、喃喃自語…… 甚至有人在撕扯自己的衣服和頭髮。 　　課程名稱：探索 　　課程：3個晚上加2個全天，近40小時 　　學費：3,300元(約台幣18500元) 第一堂課 我2006年12月份就繳了學費， 但當月的探索班已滿。 直到2007年1月24日， 在漫長的等待和期盼之後， 我終於可以去上這個充滿神秘和神奇力量的「個人才華提升」課程了。 從一些細節來看， 這門課程的確與眾不同： 它設計的學員登記表極為完備， 而且還制訂了非常細緻、表述嚴謹度近似法律文書的“12條守則”， 內容包括不能遲到早退、不能洩漏課程內容等。 朋友Z先生事先就再三提醒過我， 要提前處理好所有事務， 安排好上課時間。 因為整個上課期間必須出席所有課程， 否則只能從頭再讀， 且這種重讀的機會只有一次； 並且「邀請你暫時放下你心裡所有的事， 放下所有的分析、評估和判斷， 全身心地投入到對課程的體驗裡去， 這樣你才能有最大的收穫…” 一切都讓我對這個課程充滿好奇， 也做好了「一定要好好學習」的準備。 傍晚6時30分，我們開始報名。 十來個佩有胸牌的人在幫我們核發銘牌、發票。 他們都是老“LP”， 或正在讀第三階段的準“LP”， 回公司來做“義工”。 他們個個精神抖擻， 標槍般立得筆直， 挨個提醒新學員們寄放物品、關掉手機。 一切井然有序。 教室是一間有100多平方公尺的全封閉式房間， 沒有一個窗戶， 剛進入時感覺空氣很混濁。 教室中，100多張凳子排得整整齊齊。 我不由自主想到了軍校。 「同學們，晚上好！我叫XXX，是你們探索班的導師！」 一個40多歲的中年男子手持麥克風， 快步走上了講台。 他是中國人，來自香港，英文名叫愛德華。 他兩手空空，沒有帶講義和教案。 他將憑一張嘴上完這3晚2天的課程？ 愛德華開始向大家解釋什麼是「探索」課程， 什麼是「全體驗式」課程， 並叫學員們回答來到「探索」課堂是為了得到什麼樣的收穫。 「你希望透過課程改善你的人際關係？ 我說你已經沒有希望了！ 因為你只是希望嘛， 而不是真的想要…注意， 這個『我希望…』它沒有能量… 你抱的心態不一樣， 你最終拿到的結果也會不一樣……」愛德華說。 他要求我們發言時不能採用“我希望…”的語式， 而改為語氣更強烈的”我要…” 　　學員們早就做好了全心投入的準備， 加上導師的確有非常強的掌控能力， 因此我們的思維很快就跟著導師的引導往下發展。 「我要改善我的人際關係！」 「我要賺很多的錢！」 「我要一個健康的身體！」 學員們照著這個句式一個接一個地站起來振臂高呼， 愛德華則不停地在旁表示肯定和鼓勵。 課堂氣氛很快變得充滿激情。 隨後愛德華介紹小組長們給我們認識： “他們都是老LP，無償地回來做奉獻，支持大家！” 12個小組長上台一字兒排開， 挨個介紹自己的姓名班級， 以及自己承諾將以怎樣的態度來「支持大家拿到收穫」。 我注意到，他們多數都係商界成功人士， 另有少數來自律師、教師等崗位， 還有一個是大學的在校生。 他們的發言都很簡短有力，激情四射。 愛德華告訴大家， 在兩三個小時之後， 會給大家一個“選擇的機會”， 願意留下來並相信自己“能拿到自己想要的收穫”的就繼續留在課堂， 反之則可以退費離開。 有學員舉手說公司業務繁忙， 平常兩支手機都不夠用， 更不能隨意關掉手機。 愛德華幽默地說： 「我很理解你。 我想這世界上最忙的人應該是比爾蓋茲。 他可能全身上下都掛滿手機……」 大家一陣哄笑， 笑聲中，都自覺地關上了智慧型手機. 愛德華的口才很好， 肢體語言豐富， 在滔滔不絕地講解的同時， 十分注意和學員間的互動， 偶爾用水彩筆在講台上的展板上寫下幾個關鍵詞。 接下來， 他主要講解了「人的特質」、 「人生所有的結果都是由『看法+行動』或『意向+方法』得來的」等觀點， 並一一結合現實舉例： 「現實生活中，缺點為什麼總是難以改變？ 事後可以幫你推卸責任， 例如你可以在事後說那是我一時衝動…… 課程的觀點，人是沒有缺點的， 只有特質，你喜歡的或是不喜歡的特質……” 有些人退出了 中途有一個15分鐘的休息。 我和幾個沒有來得及吃晚餐的學員衝到樓下快餐店去吃快餐， 結果遲到了幾分鐘。 一臉嚴肅的小組長們把我們擋在了門外， 把我們一個個輪流叫到一邊談話： “你有沒有註意到你破壞了你的承諾？” 我說：“我今晚沒吃晚飯，我有胃病，剛才吃快餐時以為來得及……” 「那隻是你的藉口！ 你既然承諾了不遲到， 無論什麼樣的原因你都該自己處理到。 想想平時在現這生活中， 你是不是也是這樣輕易地破壞你的承諾？” 我一愣，意識到小組長是在藉題發揮， 啟發我去反思自己平常的一些習慣。 我對這門課程頓時有了好感。 難怪朋友們會說這課程“與眾不同”， 它和我們平時熟知的照本宣科的授課方式的確完全不一樣。 它會利用一切事件「引導」你從另一個角度去思考。 最後小組長叫我重新承諾再也不遲到後， 開門就讓我進去了。 兩個多小時很快就過去了。 愛德華告訴大家， 接下來將由他的助手、副導師L先生宣讀課程的“12條守則”， 如果大家有什麼不明白的可以提出來。 不願承諾遵守這些守則的人可以退費離開。 空出來的名額將讓給其他已繳了費的候補學員。 L先生在進入課程前的身份是成都市某公安分局的警官。 他宣讀得極慢， 且每條守則均會重複數次。 這立刻引發了一番爭執和騷動， 有許多學員質疑導師是在故意發水拖延時間。 “我們每個人每個小時付出的學費值100元， 120個人就是12000元。 我們花了12000元來聽了1個小時守則， 太不划算了！” 學員們議論紛紛，課堂秩序大亂。 對這樣的場面， 導師和小組長們似乎已司空見慣， 並沒有理會而是繼續。 有20多名學員乾脆跑到教室外的休息區喝水休息， 以示抗議。 我對在一旁旁觀的愛德華說： “導師，這些守則大家報名時就書面承諾過了， 你這樣安排很容易讓大家誤會是在拖延時間。” 愛德華掃了我胸前的銘牌一眼， 就將視線轉往了他處， 禮貌而冷淡地說了一句：「謝謝你的建議。」 我感覺他很敷衍， 再次重複了一次自己的觀點。 他沒有說話， 也沒有任何表情， 眼睛仍盯著他處，不再理我。 我覺得他對我的話很不高興。 這和我原來設想的導師的形像有些差距。 這讓我有些失落。 最後去留已明。 有人退費，有人補充。 教室裡還剩下了111人。 而意外的是， 一些原本宣稱感覺「上當了，不值得」強烈要求退費的學員又出現在了教室裡。 坐在我旁邊的建築商方先生幫我解開了謎底： 當他去退費時， 公司電話通知他的「感召人」來做他的思想工作： 「我那朋友幾乎是聲淚俱下地求我留下來， 最後乾脆說我要不上完這個課， 她就再也不理我了。 我本來猜想「選擇是否繼續上課」這個環節， 是課程設定的第一個「過濾」環節， 一些不適應這種授課方式學員將被分離出課堂， 以保證授課效果。 但聽了方先生的話後我又有些生疑， 感覺這家公司和任課的導師， 並非像我的朋友們說的那樣“無私奉獻”， “純粹是為了提升國民素質”， 而是有著明顯的經濟訴求。 後來當我得知這位愛德華導師和另一位來自香港的溫蒂導師就是公司的老闆時， 更加深了我這種感覺。 此前，課程在我心裡曾被“拔高”， 至此方回到其“商業課程”的本來位置。 我不再激動、崇拜， 而是抱著平常心繼續上課。 第一晚一直上到了凌晨零時過。 我感覺課堂上宣揚的那些理念有一定道理， 導師講得也很生動， 但心裡卻不覺得學有所值：這些內容也不是什麼新東西了， 現在書店中的「成功學」、「勵志「類書籍多有涉及， 而且透過自己的生活積累， 也能得出類似的人生體驗， 哪裡值得了以每小時100元的代價來購買呢？ 蘭女士事件 第二晚， 課堂上發生了後來據說是破天荒的意外： 導師提前結束授課！ 當晚包括我在內， 有30多人遲到。 所有遲到者被罰站立， 解釋遲到原因並被要求一個個當面做出守時承諾， 否則只有下車。 我解釋說： 「我下午6時才能下班， 半小時內趕不到。 我要么只有工作早退， 要么就只有上課遲到， 魚肉和熊掌沒法得兼。」 其實，我當時是故意這樣說， 想看看愛德華會如何引導學員處理這種兩難。 愛德華開始認為這個話題很好， 並馬上藉著這個話題講到生活中的很多兩難： 「生活中我們常常會以為只能得到魚肉和熊掌， 其實不是這樣的……」 他讓我們要盡量去做到“兩全其美”， 但卻並無具體一點的指導性意見。 我說：“我找不到兩全其美的辦法怎麼辦？” 愛德華把皮球又踢了回來： 「我沒有什麼可說的了， 你是對的！ 你都認定你自己做不到了， 誰還能夠幫助你？」 我覺得這樣的引導有些唯心， 就繼續追問。 沒想到愛德華突然生氣了， 竟當眾提出要我出局： 「李小姐，對你我可以例外。 如果你現在選擇退出， 我會破例退你全款。」 我說：「我感覺你在威脅我！ 就算我笨一點，領悟不到， 你也不能剝奪我提升的機會。 因我的“挑釁”，愛德華已有些不快。 此時一位姓蘭的女士又在無形中火上澆油： 蘭女士第一晚因身體不適早退了半小時， 今天一來就被小組長們擋在了教室門外， 告知她只能選擇另一期探索班重頭開始。 蘭女士不願意重讀， 強行衝進了教室與愛德華理論。 兩人對峙了數分鐘， 導師的臉色漸漸變了， 最後憤而退場。 臨退場前他留下一句： “你們什麼時候處理好了這件事，我們再繼續。” 愛德華一出門， 課堂立刻炸了鍋。 一位王姓女學員突然站了起來， 用手一指蘭小姐， 聲色俱厲地喝道： 「把她抬出去扔了！」 此人的身份是一個導遊， 從一進入課堂， 她的情緒就變得很容易激動。 幾十個學員圍了上去聲討蘭女士。 教室裡亂哄哄一團糟。 我突然發現，雖然才上了一天課， 課堂上宣揚的理念聽起來也很積極向上， 但課堂上不知為何已有了一種非理性的狂熱氣氛。 過了10多分鐘， 蘭女士和公司終於協商好了。 已恢復平靜的愛德華重新出現： 「蘭小姐，你可以繼續聽完今晚的課， 也可以參加一會兒的遊戲。 今晚這幾個小時算我免費送你。 但你明天不能再來這裡。」 他強調了兩次“免費奉送”， 台下有人小聲說：“太商業了！”  紅黑遊戲 接下來我們玩了課程中第一個遊戲：紅黑遊戲。 愛德華把學員分成兩組， 事先要求累積到最高的正分算贏。 結果兩組的學員都害怕對方的分數比自己高， 不約而同採取了和對方同時得零分的方式。 遊戲結束後， 愛德華做總結， 分析大家遊戲時的心態， 講解課程所提倡的觀點， 並引申到現實生活中的相關現象： 「有人做到了雙贏， 有人卻選擇了兩敗俱傷…… 你平時是否也在不停地玩這種遊戲？ 充滿「恨鐵不成鋼」的味道。 這讓剛剛還在遊戲中「勾心鬥角」的我們非常自愧自悔。 緊接著，愛德華突然宣布今晚的課程至此提前結束： 「請大家閉上眼睛， 我放一首歌給大家聽， 聽完後請大家不要說話， 靜默離開。」 (後來有小組長悄悄告訴我們： 「這是這麼多年來， 導師第一次提前下課！ 你們這個班的學員太強了， 居然弄得導師有點下不了台！」) 負責音控的LP放了一首激昂的外文歌， 愛德華拿話筒同聲翻譯： 「你是否一直在玩著同樣的遊戲？ 真的話從來不說， 說了的，也沒有幾句是真的… …」 歌詞和配樂非常有感染力， 我聽到周圍有人開始小聲啜泣。 我感覺到這個遊戲的確促人們反思自己以往的作為， 有其積極作用， 但不知為何心裡卻有些「不舒服」的感覺。 後來，我和一些學員私下交流時找到了原因。 我的不舒服來自於課程這種設計有一種很強的讓人「自我否定」的力量。 課程提倡「信任他人、真誠付出」本身並沒有錯， 但並不是所有的「虛偽」都應當受到譴責、批判和否定， 一廂情願的不設防的「真誠」同樣潛在危險。 好比一個硬幣，課程只強力展示了其中的一面， 沒有給人留下理想狀態和現實環境間的道德緩衝空間， 這會讓一些思辨能力弱的學員將自己以往的作為「全盤否定」。 【心理練習“廢墟”】 第三天晚上一上課， 導師先讓大家輪流分享紅黑遊戲的體驗。 很多學員舉手爭搶「分享」機會。 「分享」時，他們個個都很激動， 反思著自己以前的自私、冷漠、不擇手段， 承諾以後要“真誠、信任、付出、關愛-…” 這種追求「真善美」的場景很感染人， 很多人感動得熱淚盈眶。 前兩天學員累積的對課程的不滿情緒， 至此消解了大半， 大家的態度都變得「配合」起來。 課程的安排很精妙。 在經過兩天的鋪墊之後， 當晚， 課程中增添了新的重要內容：心理練習。 第一個心理練習名叫「廢墟」。 導師讓所有學員坐在地上，閉上雙眼。 伴隨著室內燈光的逐漸黯淡， 音樂聲也緩緩響起。 導師則手持麥克風在室內走動， 口中開始引導大家的想像： “現在，你來到了一座城堡前……” 導師的聲音時高時低，飽含激情， 而音樂和燈光也伴隨著他的話語適時做著調整。 當現場的氣氛慢慢醞釀出來之後， 導師的話語變得更煽情。 他用了很長的排比句， 羅列現實生活中存在的種種傷害與被傷害的狀況， 種種被毀滅的美好與希望： 「是誰，是誰對你說他會永遠愛你？ 是誰，是誰一次又一次地欺騙了你……」 而總有一種能與人自身心底埋藏的傷痛契合， 於是學員們的情緒被引發了。 　　 教室中開始傳出抽泣聲， 音樂聲則更悲壯激昂。 導師完全沉浸在自己的講敘中，聲淚俱下。 部分男女學員已情緒失控， 搥胸頓足，嚎啕大哭，呼天搶地，喃喃自語…… 甚至有人在撕扯自己的衣服和頭髮。 　　 練習結束，燈光大亮， 滿室都是紅腫的眼睛和濕漉漉的臉龐， 大家相互對視，恍然如在夢中。 導師要求大家不能交談，靜默離開。 小組長們則在門口一字兒排開， 手上都捧著紙巾盒，任人取用。 　　 此時，我對這個課程更加質疑。 練習極力煽情， 讓大家回憶起種種不幸和痛苦， 卻不引導大家以一種積極的心態去回顧自己的過往經歷： 坎坷和曲折也是人生財富！ 而這個練習顯然採用了極高明的心理技巧， 類似「群體催眠」。 可班上的111人不同性別、年齡、職業， 心理健康狀況也完全不同， 課程用強有力的心理技巧攪起大家心底的沉渣， 卻讓大家自行去“重新沉澱”， 這其中顯然存在隱患。 事實證明了我的擔心不是多餘。 後來在報道這一課程時， 很多讀者給我打電話， 提供了多起學員上課後精神異常的案例， 甚至有一個福建學員上課後竟赤腳跑到火車北站撞火車自殺身亡！ LP行 (lp:讀完了三個階段課程的學員) 接下來的兩個一整天， 課程的主要內容仍然是由導師解釋一些概念， 再結合一些遊戲或練習， 「引發」大家的「體驗」或「思考」。 其中有許多方式都含有許多心理技巧。 例如讓我們學員彼此目光對視、擁抱、閉目想像、在黑暗中發洩… 　　 我看到了一些微妙的變化， 部分學員變得亢奮、激進，思維單一。 而有半數以上的學員的感受是： 有一定收穫，但和昂貴的學費相比， 並非物有所值。 　　 在第5天下午， 課程安排的最後一項是一個很莊嚴的儀式“LP行”： 所有的新學員手拉著手貼近牆壁站成一個圈， 激動人心的音樂聲中， 此前各個班的老LP們手牽手一個個走進教室， 從我們每一個新學員面前緩慢走過， 與我們每個人目光對視， 再手拉著手一個個走出教室。 期間導師會一直手持麥克風引導我們去「體驗」。 　　 隨後，愛德華要大家閉上雙眼，靜默等待。 他則繼續伴隨音樂聲引導著我們的思維： 「經過這幾天， 你看到了什麼？ 體驗到了什麼？ 你和來這裡之前， 有了怎樣的不一樣？ 還記得你第一天來上課時的情形嗎？ 教室裡再次傳出了哭聲。 我也不受控制地鼻子泛酸。 當導師要我們打開雙眼時， 我意外地發現我的朋友Z先生和那位在茶樓裡與我爭論過的老LP都手捧鮮花站在我的面前。 驚喜之餘， 我的眼淚一下子湧了出來。 　　 不只是我，每個新學員一睜開眼睛， 看到的都是一束美麗的鮮花和鮮花後面感召人那熟悉的笑容。 於是，尖叫，擁抱，哭泣…… 整個人群都在瞬間陷入了激情的海洋， 認識的、不認識的都在相互擁抱。 迎新生活面談會 短暫的休息之後是一對一的「迎新生活面談會」。 公司認為，至此學員們已經「新生」。 　　 幾個LP手牽手站成一個大圈， 我們則手牽手組成一個內圈， 大家緩慢地順時針轉動， 選擇面談對象。 我選了一個留披肩發的王小姐。 後來我得知她是小組長丁先生的妻子。 她們夫妻倆都在上課後決定回來「做義工」。 　　 一切程式嚴密設計，精確到分秒， 副導師L先生隨時手持秒錶， 限定交流題目和交流時間， “這幾天的收穫，開始……停！”， “你的目標，實現目標的障礙，開始……停！ 　　 那時，我覺得自己很像一個被操控的機器人。 我說了自己對課程的質疑， 王小姐的反感明顯寫在臉上。 她用了和我的朋友Z先生他們一樣的表情和語氣： “你太自以為是、太固執！ 你已經沒有接收和提升的空間了！” 而讓我意外的是， 所謂的「迎新生活面談會」其實最終目的竟是讓大家繳錢讀二階段！ L先生嚴格限定我們的談話時間， 一個話題往往只準談幾分鐘， 卻把大量時間留給了LP們說服學員們繼續就讀。 　　 王小姐開始認為我不會讀二階段了， 已經懶得搭理我， 當我表示我要讀二階段時， 她的態度一下子積極起來， 把我直接帶入另一間封閉式教室， 填「突破班」的報名資料， 並叫我立即付款。 　　 我表示沒有帶夠錢， LP們要求我先付預付款、承諾付全額時間， 並親筆將「自己的承諾」當眾書寫在展板上。 這種「現過現」甚至有點變相強制的感覺和先前課程宣揚的理念格格不入， 我感覺很不舒服： 「我已經報了名了， 說好了明天會來付款， 為什麼還要書面寫下來示眾？ 你們不是要求我們'要信任他人'嗎？ 　　 馬上就有幾個LP湊上來， 義正辭嚴地開導我： 「這不是不信任你， 也不是強迫你， 而是為了鍛鍊你的嚴謹性， 鍛鍊你履行諾言的能力！ 你既然承諾了明天付全款， 為什麼就不敢寫下來？ 不要以為只有在課堂上才是上課， 這個課程的特別之處正是在於什麼時候都可能是學習機會， 這也正是課程的內容之一！ '突破'已經開始了！ 　　 我覺得他們有些強辭奪理， 可是一嘴難敵眾口， 我還是乖乖地閉了嘴， 老實地在展板上寫下了自己的姓名和承諾明天繳全款等內容。 　　 111人中約有40餘人當時就決定了繼續讀「突破」。 　　 畢業典禮風波 　　 1月30日， 公司為我們舉行了「畢業典禮暨嘉賓會」。 主持人也由LP義務擔當。 參加者除了學員， 還有學員們帶來旁聽的「嘉賓」。 　　 畢業儀式搞得很莊嚴神聖， 主持人要我們閉上雙眼， 小組長將送給大家一份「珍貴的禮物」。 在閉目等待的過程中， 主持人不停地強調這份「禮物」的重要性以及它將為我們的生活帶來怎樣的不同。 小組長將禮物輕輕放在我們手心， 再鄭重其事地幫我們握緊手掌， 貼在胸口。 我感覺到那「禮物」似乎是個信封。 而當我們睜開眼睛， 發現信封中裝著的原來是公司給我們頒發的「畢業證書」。 　　 有的學員深受感動， 緊緊將證書緊貼在胸前，滿面激動， 彷彿真地從此掌握了一種神奇的力量。 　　 領了證書之後， 主持人開始邀請學員們上台「分享」自己上課後的收穫。 此時，學員們已有些身不由已了， 大家都不約而同地「只說好的，不說真的」。 「課程改變了我的一生」、 「課程讓我擁有了改變命運的力量」…… 種種誇張的說法， 和平時學員們私下的真實交流完全不一樣。 　　 「分享」之後， 就是針對嘉賓的「感召」了。 主持人在展板上掛了一幅抽象的畫， 讓賓客們來看這幅畫畫的是什麼。 每個人的看法都不一樣， 主持人趁機開始遊說嘉賓們也來上這個課程， 「讓自己能多一些角度看問題」。 　　 「我抗議！商業味兒太濃了！」 一位鄒女士突然站了起來， 批評公司的「口是心非」： 「前幾天我感覺還挺好的， 覺得你們在教我們真善美， 可是你們現在卻赤裸裸地曝出了商業目的！ 　　 現場頓時亂了套。 有學員上台支持鄒女士， 有LP上台支持主持人。 大家爭執不休， 最後畢業會只得草草結束。 鄒女士本來已經繳了二階段的學費， 至此強烈要求退費。 有些學員也隨聲附和要求退費。 原來，他們和我一樣， 也對「迎新生活面談會」後公司催款的一系列做法很反感。 　　 鄒女士是商界成功人士， 業務涉及名牌代理商、通訊設施等。 她邀約了10餘位學員到公司樓下的茶樓「交流」。 最終，鄒女士和一些已繳了二階段學費的學員選擇了退費。 到二階段突破班開課時， 只剩下了29人， 其中還有3名是其它探索班的「插班生」。      第二階：“你去死吧” 　　 一個同學從我面前走過， 多數人對我說「你去死吧！」 甚至有人叫不出我的名字， 「我不關心你，你去死吧！」 也有人鄭重地投我一票， 眼中滿含深情。 每一個人走過我的面前， 我的心都要承受一番煎熬。 同學們的心情和我一樣， 每個人都哭得很厲害。 　 ……這時我很奇怪地發現， 我在課程上「拿到的能量」已經消失無蹤了。 我覺得自己像做了一場夢， 夢裡說了許多不該說的話， 做了許多不該做的事。 我的心情很複雜，充滿迷惘。 而其他同學也都有類似的矛盾心態。 　　課程名稱：突破 　　課程：5個白天4個晚上 　　學費：6900元(約台幣36000元) 　 「突破」課程有5天4夜， 期間所有學員不能回家， 公司在玉林生活廣場旁的一家賓館為我們安排了住宿。 　　 「突破」的授課方式和「探索」大同小異， 也由導師講授、學員分享、做遊戲與練習等組成。 但其具體內容的力道與強度和「探索」簡直不可同日而語。 導師的態度也由「探索」時的「溫和」變得極為「尖銳」。 而讓我最意外、最尷尬的是， 「突破」導師正是我的朋友、曾任成都某局長的X小姐。 　　 在課堂上，我看到了一個與平常完全不同的X。 她態度嚴厲，措詞尖刻，有時還會爆粗口。 第一天下來，很多學員在背後罵她。 但我卻知道，她其實是個很善良、有愛心的人。 她這麼做，是因為她堅信只有透過這種方式， 才能讓學員們“突破自我”， 她堅信她所做的一切都是為了我們好。 　　 那5天4夜，完全是一種魔鬼式訓練。 課程安排得極為緊湊， 時間與正常作息錯亂： 每天從早上10時上至晚上12時過， 期間只在下午四五點鐘安排一次就餐時間。 每次課間休息時間很短， 且要完成「作業」。 夜間下課後還有大量作業。 課程的原則是「每晚至多把頭放在枕頭上4小時」。 　　 這樣的「魔鬼式訓練」讓我們處於極度疲憊之中， 腦子常常“轉不過彎”， 加上長時間身處一個封閉式環境(教室無一個窗戶，空氣混濁)， 更容易被動接受外來資訊。 在「突破」結束當天， 我們全班同學無一例外全部聲音嘶啞，雙目紅腫，體力透支。 隔日幾個學員病倒， 我和另一名都江堰的男學員則因肺部感染不得不到 醫院輸液治療。 「突破」課程中的一些遊戲會採用故意侮辱人格的方式， 這很考驗人的承受能力。 第一天晚上， 一個練習讓我印象至深： 學員們組成馬蹄形， 每個學員輪流站在馬蹄形缺口處的“壓力點”反思自己， 其餘學員們則一個接一個地反复批評， 直到X導師叫停為止。 如批評得不夠大聲， 會被認為是對死黨們不真誠， 甚至遭到X導師痛罵： “我X你！你到這個時候都還在出賣你的朋友、出賣你自己！” 　　 「你虛偽」、「你懦弱」、「你自以為是」…… 當陷入了眾口一詞的批判浪潮中時， 站在「壓力點」的人不管男女老少， 也不管職業如何， 哪怕一些現實生活中身份顯赫的「成功人士」都會感到惶恐， 身體不由自主發抖， 甚至淚流滿面。 　　 我剛開始時也被那種氣氛嚇倒， 心裡一直打鼓，小腿不停打閃。 但過了一陣， 我發現每個學員得到的批評都是一模一樣的， 就慢慢鎮定下來。 　　 輪到我去接受批判時， 我把這當成了考驗自己心理承受能力的挑戰。 我沒有哭，也沒有垂頭喪氣， 而是挺胸抬頭和每一個批判我的人目光對視。 他們不停地批判我「高傲」、「自以為是」、「冷漠」、「麻木」…… 說實在的，我當時確實已經有點「麻木」了， 對他們的批判只當是耳邊風， 沒有了多少感覺。 　　 後來X導師走了過來， 不是像同學們那樣胡亂批判， 而是根據她對我的了解， 說著我平時生活中的種種狀況， 她說得很動情，聲淚俱下， 句句都打在我心上， 我終於忍不住流下了眼淚， 但我仍然明確表示拒絕接受這種「非理性批判」。 X很失望，有些恨鐵不成鋼」說： 「你完全不是在體驗，你還在固執於你的看法！ 」 　 「救生艇」練習 第二天晚上有一個「救生艇」練習， 裡麵包含的「死亡體驗」被外界傳得十分玄乎， 以致引來一些媒體的批判。 而對我來說， 這是整個學習期間感受最深的練習， 我的思想受到極大衝擊， 最終情緒失控。 　　 在進入練習之前， 有很長時間的鋪墊。 X導師叫我們閉目坐在地上， 開始引導我們的想像。 她先為我們描繪出一幅幅溫馨、美好、平和、真實的生活場景， 每個細節都刻畫入微： 「……你上了這個遊輪， 你找到了自己的房間…… 你收到了一張請柬， 原來是船長邀請你參加晚上的PARTY……」 我們不由自主地入戲， 彷彿真的坐在一艘豪華遊輪上， 在大海裡暢遊。 　　 「嘩啦！」 突然，耳畔響起兩聲驚雷般的巨響， 我們被嚇得驚叫。 X的聲音變得緊張： “我們的遊輪撞上了冰山， 很快就要沉入海底。 船上的救生艇只能乘坐4個人…” 　　 她要我們睜開眼， 一個個誠實地表態， 是否選擇去上這個救生艇， 並說出理由。 結果選擇「上」與「不上」的人基本上各一半。 我選了「不上」。 　　 X對我們的選擇不置可否。接下來是投票表決誰可以上船。 　　 X叫我們站起來挨牆壁站成一個圈， 取下掛在胸上的寫有自己名字的銘牌， 然後給我們每人發了四根木簽， 每根簽代表救生艇上的一張船票。 具體規則是： 我們每個人依順時針次序， 從每一個同學面前走過， 然後對每一個同學做出選擇， 是否把「船票」送給對方。 　　 如果你送了， 要說：「XX，我投你一票，你可以上船！」 如果你叫不出對方的名字， 要說：「雖然我不認識你，但我還是決定投你一票，你你可以上船！ 如果叫不出對方的名字， 則說：「我不關心你，我不記得你的名字，我不投你這一票，你去死吧！」 對方則要叫著自己的名字大呼一聲「 XX死了！ 那時誰也不覺得這只是一個遊戲， 我們都面臨著真實的殘酷的選擇。 29個人，卻只有4張票、4個求生的機會。 我的目光飛快地從所有同學的臉上掃過， 暗暗做著選擇。 但這樣的選擇實在太難， 看著一張張熟悉的臉孔， 我怎麼也難做取捨， 還沒開始投票已經忍不住淚流滿面。 全班同學無一例外，都開始哭泣。 　　 一個同學從我面前走過， 多數人對我說：「你去死吧！」 甚至有人叫不出我的名字， 「我不關心你，你去死吧！」 也有人鄭重地投我一票， 眼神中滿含深情。 每一個人走過我的面前， 我的心都要承受一番煎熬。 同學們的心情和我一樣， 每個人都哭得很厲害。 　　 輪到我投票時， 我簡直是個步履維艱。 我想投票的同學很多， 有的是我喜歡的朋友， 我對他有情； 有的是已經給我投了一票的朋友， 他對我有恩； 有的是開課時指派給我的“死黨”， 我對他有責任……4張票根本不夠。 我的思想不停地激烈交戰。 　　 最後我把票投給了兩位我最喜歡的同學和兩位我最同情的同學。 當我把票都投完， 我只能對剩下的同學說：「你去死吧！」 我也曾試著解釋一句： 「不是我不想救你，我已經沒有票了…」 而做「支持」的小組長則在旁冷酷地說： 「你真虛偽！別再說好聽的了！ 你明明有4張票，卻沒有選擇給他一張。 你要真的看重他，說什麼也會給他留一張票。 　 這些話很刺我的心。 是的，我不是“沒有辦法”， 而是“已經選擇”， 即使這個選擇我很不情願。 我能領會到這個遊戲的寓意。 在現實生活中， 我從來沒有遇過這樣「生死抉擇」、 面對自己內心的時刻。 　　 最後清點票數， 我得了8票，是得票最多的4個人之一， 有一個同學一票也沒有。 　 「好好想一想吧！ 你平常都對你的死黨們做了些什麼？ 為什麼會是這樣的結果？ 他們為什麼要把這麼寶貴的生的機會留給你？ 他們為什麼不肯投票給你？… …” 　　 大家都哭著開始反思， 我也想到了很多。 然而遊戲至此並未結束。 X導師讓我們按得票的多少排列好坐在地上， 「海難」還在繼續， 依票數從少到多， 25個同學陸續被「拋」進了「海裡」 ——在教室的地毯上躺下。 　　 而我們4個得票最多的人， 誰都沒有自己投一票。 X導師掀翻了象徵著「救生艇」的4張椅子： 「那麼多人愛你們，信任你們， 把生的希望留給你們， 你們卻辜負了他們， 你們自己都不想活下去， 再多的人要你活下去也沒有用啊……」 最後，我們也都被「拋進海裡了」。 　　 X導師要我們閉上眼睛， 繼續引導我們的想像： 「海難的消息傳回了你所在的城市， 你的親人焦急地湧向碼頭…… 人們終於盼來了救生艇， 可是沒想到上面是空的， 一個人也沒有…… 你的父母親在哭， 哭他的孩子…… 你的丈夫、你的妻子在哭…… 你被裝進了棺材…… 你被放進冰冷的墓地……” 　　 X導師詳細描述我們的葬禮， 描述我們親朋好友的悲痛， 話語飽含感清，聲淚俱下。 我們已經完全沉浸在那虛擬的死亡之中， 除了眼淚橫流， 一動也不能動… 　　 中毒 　　 當天晚上，我的感觸特別深。 我突然改變了自己以往對生命、對生活、對感情的許多看法。 我終於明白這個課程為什麼會讓那麼多人著迷了， 它採用的心理技巧力度很強， 你根本無法抗拒。 　　 自此，我先前對課程累積的負面印像已經蕩然無存。 我整個身心都沉浸在課程的那種氣氛中。 晚上一下課，我就激動萬分地給我老公打電話， 要他也來上這堂課。我老公覺得我莫名其妙， 笑著說我可能「中毒」了，我卻渾然不覺。 那時，我覺得課程的某些設計真是非常獨特， 花6900元真的值。 　　 二階段結束時， X導師勸我們報名上三階段： “領導鍛鍊其實是整個課程最重要的部分， 尤其是第三個週末， 會讓你感覺像新婚一樣快樂…” 　　 我毫不猶豫地繳了費， 並說服了3個還有些猶豫不決的同學繼續就讀。 X導師和小組長們不停地「嘉許」(課程術語)我， 說我「變化很大，能量十足」。 我當時感覺也非常良好， 連自己已經上呼吸道感染並引發肺炎都沒有察覺。 　　 上完課的第二天， 我身上的熱情突然如潮水般退去。 我一下子病倒了， 連說話都困難， 只好住進了空軍醫院。 　　 等我出了院，已經是春節了。 這時我很奇怪地發現， 我在課程上「拿到的能量」已經消失無蹤了。 我覺得自己像做了一場夢， 夢裡說了許多不該說的話， 做了許多不該做的事。 我的心情很複雜，充滿迷惘。 而其他同學也都有類似的矛盾心態。 大家互相發著短信， 內容都是「我感覺能量正在慢慢消失…… 我又變得和以前一樣了……」 有一個同學則發短信說： 「我好像已經不能適應這個社會了， 我的情緒很低落…” 　　 春節期間， 我反覆回想課程中的點點滴滴， 感覺這個課程有一定積極的功效， 但又存在著一些重大缺陷， 可是我的腦子成了一團糨糊， 我一時也想不明白它的問題究竟在哪裡。 　　 春節後，我開始上第三階段。我所在的班級編號為LP56。 【3C就是充滿激情地去拉客】 第三階 3C就是充滿激情地去拉客 烈日下，大批被風暴拋上沙灘的海星即將悲慘地死去。 一個來海邊遊覽的小女孩不停地撿起海星拋回大海。 母親勸她說，這麼多海星，你永遠撿不完。 但小女孩說：“能救一個是一個！” 　　 等到「畢業」時， 我所在班級共做成了20多個「3C」── 「海星」們均是學員們的配偶、親人、同事、同學和下屬。 有些「小女孩」為了完成任務， 甚至不惜自己掏錢來發展「海星」。 　　 有的學員感覺自己從中得到了提升： 學會了溝通，重新審視了自己和他人的關係……； 而有的學員則為此付出了慘痛代價： 夫妻不合、父子不合、朋友反目…… 　　課程名稱：領導鍛煉 　　時間：3個月，每個月安排一個週末上課，平常每個週末則由小組長組織開會 　　學費：3900元(約台幣19000元) 　　做3c 讓我驚訝地是， 整個三階段基本上只圍繞著一個主題在進行： 做3C！即感召自己的親朋好友也來上課， 必須做到「讓感召對像有意願提升自己、完成作業、繳錢」的「三個認證標準」。 　　 第一個週末完全是做3C。 　　 課程中，“感召者”被稱為“小女孩”， 被感召者被稱為“海星”。 這來自一則寓言： 烈日下，大批被風暴拋上沙灘的海星即將悲慘地死去。 一個來海邊遊覽的小女孩不停地撿起海星拋回大海。 母親勸她說， 這麼多海星，你永遠撿不完。 但小女孩說：“能救一個是一個！” 　　 導師名叫溫蒂，來自香港地區。 她也是這家公司的老闆。 她告訴我們， 你就是那個“小女孩”， 你的親朋好友就是“海星”。 他們正在現實生活中“擱淺”，亟需你的“挽救”。 惟一的挽救方式就是── 「感召」他們也來讀這個課程！ 溫蒂說： 「做3C其實是件利人利已、功德無量的事， 甚至可以說關乎我們整個國家民族的興亡。」 她要求我們每個學員每週都必須宣言自己下週要做到多少個3C， 否則只能「下車」。 　　 剛開始時，班上大多數學員都拒絕宣言。 溫蒂將我們的拒絕的原因歸結為「你沒有自信」、」 你看扁你的朋友，也看扁你自己」、 「你沒有愛心」、 「你不關愛你的朋友」、 「你不負責任”等等。 我表示異議， “鍛煉的方式有很多種， 為什麼一定要做3C？” 　　 溫迪一下子勃然變色， 聲色俱厲地喝道： “你既然選擇了我這個教練， 就必須服從指揮。 這是一場遊戲， 你必須遵守起碼的遊戲規則， 不然你就不要玩兒這個遊戲。” 　　 至此我明白了，在「領導運動」階段， 去發展新學員不是聽憑自願， 而是一種強制要求。 而任何“商品”都不能強買強賣，尤其是“精神產品”，更不能強迫他人接受。 　　 我突然從先前的渾缺失中醒悟過來： 課程的本質其實是以一些強而有力的心理手段來控制我們， 然後利用我們的熱情去為公司「拉客」。 第一階段， 課程要我們「放下分析、評估和判斷」就是放棄理性， 讓我們成為「空杯」； 第二階段， 課程摧毀我們原有的心理結構， 並在我們的「空杯」中註滿新的“水”， 其本質上是讓我們形成“課程是挽救我們生命的惟一途徑”、 ”課程至上”的觀念； 第三階段， 則是利用我們去發展新學員。 　　 仍然是“做3C” 　　 這時， 我已由一個普通學員的心態轉換成了一個記者的職業心態。 為了能繼續學完課程， 掌握更多的信息， 我做出了讓步， 而別的一些性急的學員則和溫迪發生了激烈的衝突。 　　 對這種場面， 久經考驗的溫迪早已見慣不驚。 她最不放心的是我， 沉著臉問我是不是「臥底」。 我說，我是一個記者， 但是不是“臥底”， 你可以找老LP們打聽， 我從2003年起就和你們有接觸了， 我參加過你們當年捐建鐵牛村小學。 溫蒂這才緩和了臉色， 提到2006年廣州日報的一篇報道： 「那個記者根本沒有上課， 完全是在亂寫。 大家都知道， 我們這些上課內容是不能對外公佈的， 不然會影響上課的效果。 是臥底，我會請你出去。 　　 第二個週末， 我們搞了一天戶外拓展， 第二天仍然是「做3C」。 但到第二天晚上， 我們班做的「3C」居然為零個！ 這個可是該公司成立以來破天荒的第一次。 按照課程的設計， 戶外拓展學員們都會很開心， “很有能量”， 接下來馬上去做“3C”應該很容易， 但沒想到我們居然一個新學員也沒有“感召”到。 　　 我們這種「抗拒的狀態」讓公司很不滿意。 副導師L先生最後要求我們每人必須在1週內做到1個“3C”， 未做成者“下車”。 我據理力爭： 「你以為我們只是在遊戲， 可我們實際上是一個契約關係。 事先我們的約定中根本不包括做3C， 你也沒有告知我們必須要做到3C才能上課， 現在要趕我下車， 我會將請律師到法院起訴！ 」 由於我態度強硬， 公司做出了讓步， 讓我做出“宣言”後繼續“鍛煉”。 　　 隨著課程地深入， 班上部分學員對做3C的認同度越來越高， 甚至從中體驗到了「崇高」和「滿足」。 再到後來，部分同學已經完全是「課程至上」。 3個月內， 每週學員們都要回公司開半天會。 由於有些同學在外地， 有些同學又常出差， 因此往往有人缺席。 一個自認為鍛煉得很好的同學對此很不滿， 批評其他同學“不配和我做死黨”， 並說“說好是來鍛煉自己的'無論如何'的心態的， 哪怕是家裡父母死了，也該先來開了會再說」。 　　 而我的一個朋友在LP57。 她告訴我： 「有一天開會時， 一個男學員意氣風發很有成就感地對全班同學說'今天為了來參加這個活動， 我老婆都出走了， 但我還是來了。 '說完很感動地望著大家， 彷彿自己很偉大。 　　 等到「畢業」時， 我所在班級共做成了20多個「3C」── 「海星」們均是學員們的配偶、親人、同事、同學和下屬。 有些「小女孩」為了完成任務， 甚至不惜自己掏錢來發展「海星」。 　　 3月4日， 一位女學員遭到全家人的一致反對， 4個女友也險些與她斷絕關係； 5月6日， 都江堰的一位男學員被父親打了一頓後關在家裡， 禁止他再去上課； 6月1日是學員們必須出席的第3個週末， 而有一女學員則提前4天就被男友「軟禁」起來， 手機也被砸壞。 其男友也曾是個LP， 她就是男友的“海星”， 但後來其男友卻漸漸認為這個課程有問題”禁止她再來上課。 　　 6月3日， 我們終於上完了最後一堂課。 而X導師曾說的「新婚一樣的感覺」並沒有出現。 整整三個週末， 我們都在芙蓉古城的「皇城」樓上的小會議室裡， 搞主題為「感恩」的練習， 內容仍然主要是「嘉許」、擁抱、關懷死黨…… 此時，那些曾讓我熱淚盈眶的氛圍早已感動不了我了。 　　 小組長們 　  在這最後一天， 還有一個重要的儀式是專為小組長們準備的。 　　 小組長們是課程中一個很重要的角色， 他們甘願佔用自己的工作、 生活時間花費高額成本來這裡為公司免費打工——協助導師授課。 在「領導鍛鍊」階段， 小組長的角色尤其重要， 任務繁重： 3個月內， 每一週小組長們都要組織所有學員開會、 負責收集「作業」並對每位學員進行3次「電話指導」。 其中有幾位小組長來自綿陽， 他們平均每週要自駕車在綿陽和成都之間往返兩次， 花費不菲。 　　 在儀式的進行過程中， 小組長們被別人也被自己感動得淚流滿面。 而這種「感動」正是支持他們「費錢費力打義工」的動力。 他們每個人都對我們說了同一句話： “我體驗到，付出真的是快樂的！ 死黨們，我感召你們也回來做小組長吧！” 　　 而緊接著， 副導師L先生就開始給大家發一個單子， 要我和同學們「選擇並承諾」在什麼時候回公司當「義工」。 而正是有了小組長們“接力式”的“無私奉獻”， 公司僱用的員工非常少， 節約了大量的人力成本。 這家年營業額近千萬元的公司僱用的固定職員僅有數人。 　　 上完課後， 我又去了廣州、香港對這門課程做深入訪談。 在專家的分析指點下， 我對這門課程的起源、 性質和社會影響、 存在的隱患有了更深刻的認識。 　　 6月14日， 我的調查報道見報了。 彷彿一滴水滴入了油鍋， 我的電話天天被打爆。 罵聲、恨聲、表揚聲、感激聲， 充塞我的耳朵。 　　 我告訴自己， 這只是一個開始。 對這個課程的關注， 我將繼續下去。 現在是一個價值多元化的社會， 人們有自由接受各種思潮、理念的權利。 身為一個記者， 我能盡的義務只是， 把這個課程存在的缺陷和可能帶來的危害盡可能全面地告訴公眾，以作參考。 　　 後記 　　 對不起，我是臥底 　　 發稿前，我的心情很煩躁， 莫名其妙地悲傷。 我知道，稿子一出， 我將失去這群原本志同道合的朋友了。 我很心痛。 那時， 我突然想起了《無間道》中梁朝偉扮演的臥底。 「對不起，我是一個警察！」他說。 而我是一個記者。 我給「感召」我去上課的Z先生打了一個電話， 我不知道還能說些什麼， 電話接通後， 我哭了許久才能出聲。 我說：「我知道這個課程是你們的信仰， 這個課程在成都是靠你和X姐才發展起來的。 你們信任我， 覺得這個課程對我有幫助， 才會引薦我去， 我現在卻要來毀滅這個課程， 我就像在殺死你們的孩子！ 　　 Z卻很平靜： “既然你已經決定這麼做了， 既然你認為你是對的， 那你就去做吧！ 有什麼後果都是我應該承擔的， 我該為我自己的選擇負責。” 　　 我說：“我不是認為自己錯。 我很清楚地知道我在做什麼。 我是為我將失去你們而心痛！” 　 Z說：“這件事沒有對錯， 各自的信念和角色不同。 到了這個時候， 你還能給我打這樣一個電話。 我很感謝。” 最後我說：“我是一個記者！我真的別無選擇！” Z很尖銳地說： “不，亞玲，你不是別無選擇，而是已經選擇。” 我再也無話可說。 這句話點了我的穴了， 直刺我的心。 是的， 我不是沒有選擇， 而是已經選擇。 身為一個記者， 我有義務把這個課程存在的缺陷和可能帶來的危害公諸於世。 掛了電話，我失聲痛哭。 我知道，從明天起， 我和這群朋友都已經是路人。 原文報導原文第4頁有其他記者找心理專家寫對這事件的分析 滿值得大家點進去看看了解一下參考痞課邦別人引用的文章留言上面的台幣價錢''']

In [5]:
text

["曾經我差點也被某個朋友拉去上課， 也是講一些不去上課就不會改變人生不會成功的話， 還好身邊還有幾個清醒的朋友， 直接戳破這些話術， 說如果真的想改變， 書店就很多書了， 網路上也有很多免費資源能看， 真的沒必要花這麼貴的錢去上課。 加上自己那時候剛畢業還沒找到工作也真的沒錢， 實在沒辦法做出那個拉人的朋友所說， 去借錢貸款上課的行為， 後來我就明確告訴那個朋友我的想法， 講完他還想拉我去上課， 我就不理他了。 這篇文章是我當初爬文超久才找到的， 一開始是看痞課邦有人轉發， 雖然文章年代久遠， 名稱會換來會去， 但本質上是差不多的， 換湯不換藥， 其他機構的課程也大同小異。 這篇文章皆露的內容， 對我幫助滿大的， 真心佩服及感謝寫這篇文章的記者， 我已經找到原文的完整內容了， 上面金額都是人民幣， 我有參考痞課邦寫的金額寫在旁邊簡單換算， 上面的課程金額現在至少要再翻倍， 希望能讓這邊的大家也有幫助。 ------------------------------------------------------------ 懶得看字的 建議先看這個記者於2021的專訪影片 裡面就有稍微提到2007年她當臥底的事--------------‐--------- 原文記者李亞玲 2007年於 成都商报 擔任記者 臥底半年所寫 【揭秘菁英課程生命泉源：強迫洗腦的過程】 訪談手記 好奇害死貓 第一次知道「生命泉源」是在2003年7月初。 一王姓讀者打電話給我， 盛情邀請我去參加他和他的朋友們的一次社會活動 ———選一處貧困鄉村建一所希望小學。 希望小學的校址最後選定在蒲江縣西崍鎮鐵牛村。 2003年7月19日上午8時30分， 我們在成雅高速蒲江站出口處集合， 相約前往鐵牛村商討具體建校方案。 很快就來了一個由別克、豐田、尼桑、三菱等9輛豪華轎車組成的車隊。 這些神祕人士都是三、四十歲的中年人， 有的還特地帶了未成年的兒女到鐵牛村去「體驗」生活。 這些人在成都一家公司參加了一個「很有價值的課程」。 所有學員所繳的學費中的一部分會劃入公司內部的一個慈善基金， 用於捐建希望小學。 我很好奇，可一打聽， 這個課程的學費很昂貴， 讀完三個階段(一個階段的上課時間僅幾天)需要花2萬元(約台幣10萬)左右， 也就打消了念頭。 但他們都鄭重其事地說： “學費聽起來貴，可是和你學到的東西相比

In [ ]:
text = [
    "店員服務很好。",
    "畢比登推薦，天下三絕名不虛傳！！ 牛肉麵、排骨飯、小菜、青菜都美味可口。乾淨衛生、有溫度的親切服務、親子友善、環境優美、氣氛佳。家庭或朋友聚餐首選，大推！！",
    # "B4🤭"
]
# text = ["裝潢乾淨有質感，蔬菜新鮮口味不錯。湯飯加15元可以吃到飽，素食店的價位，均消費85起。週一有九折優惠。"]

In [6]:
# nlp = Language(vocab=vocab)
nlp = blank("xx")
nlp.tokenizer = CostSegmenter(nlp.vocab)
nlp.add_pipe("tag2pos")
nlp.add_pipe("pending_aspects_opinions_pipe")

nlp_latin = load("./vec")
EBS = EmoBankSearch(nlp_latin)

In [7]:
# nlp.analyze_pipes()

In [8]:
docList = list(nlp.pipe(text))
docList_ebs = list(map(EBS, docList))

In [ ]:
displacy.render(docList[0])

In [10]:
for doc in docList_ebs:
    print(f"---doc.text: {doc.text}---")
    # print(
    #     f"---{doc._.pending_aspects_opinions_text}, {doc._.pending_aspects_opinions_i}---"
    # )
    print(f"---{doc._.aspect_sentiment_triplets}---")
    for token in doc:
        print(token, token.pos_, token.tag_, token.dep_, token.head)

---doc.text: 曾經我差點也被某個朋友拉去上課， 也是講一些不去上課就不會改變人生不會成功的話， 還好身邊還有幾個清醒的朋友， 直接戳破這些話術， 說如果真的想改變， 書店就很多書了， 網路上也有很多免費資源能看， 真的沒必要花這麼貴的錢去上課。 加上自己那時候剛畢業還沒找到工作也真的沒錢， 實在沒辦法做出那個拉人的朋友所說， 去借錢貸款上課的行為， 後來我就明確告訴那個朋友我的想法， 講完他還想拉我去上課， 我就不理他了。 這篇文章是我當初爬文超久才找到的， 一開始是看痞課邦有人轉發， 雖然文章年代久遠， 名稱會換來會去， 但本質上是差不多的， 換湯不換藥， 其他機構的課程也大同小異。 這篇文章皆露的內容， 對我幫助滿大的， 真心佩服及感謝寫這篇文章的記者， 我已經找到原文的完整內容了， 上面金額都是人民幣， 我有參考痞課邦寫的金額寫在旁邊簡單換算， 上面的課程金額現在至少要再翻倍， 希望能讓這邊的大家也有幫助。 ------------------------------------------------------------ 懶得看字的 建議先看這個記者於2021的專訪影片 裡面就有稍微提到2007年她當臥底的事--------------‐--------- 原文記者李亞玲 2007年於 成都商报 擔任記者 臥底半年所寫 【揭秘菁英課程生命泉源：強迫洗腦的過程】 訪談手記 好奇害死貓 第一次知道「生命泉源」是在2003年7月初。 一王姓讀者打電話給我， 盛情邀請我去參加他和他的朋友們的一次社會活動 ———選一處貧困鄉村建一所希望小學。 希望小學的校址最後選定在蒲江縣西崍鎮鐵牛村。 2003年7月19日上午8時30分， 我們在成雅高速蒲江站出口處集合， 相約前往鐵牛村商討具體建校方案。 很快就來了一個由別克、豐田、尼桑、三菱等9輛豪華轎車組成的車隊。 這些神祕人士都是三、四十歲的中年人， 有的還特地帶了未成年的兒女到鐵牛村去「體驗」生活。 這些人在成都一家公司參加了一個「很有價值的課程」。 所有學員所繳的學費中的一部分會劃入公司內部的一個慈善基金， 用於捐建希望小學。 我很好奇，可一打聽， 這個課程的學費很昂貴， 讀完三個階段(一個階段的上課時間僅幾天)需要花2萬元(約台幣10萬)左右， 也就打消了念頭。 但他們都鄭重其事地說： “學費聽起來貴，

---

In [ ]:
import pandas as pd
from MyUtilty.eveal import ans2aop, all_score
from MyUtilty.pat_discover_engine import PatternDiscoverEngine

In [ ]:
test_gold300 = pd.read_json("/workspaces/ABSA/test_gold300.json", lines=True)
test_gold = test_gold300[test_gold300["answer"].str.len() > 0]
test_gold["context"] = list(map(lambda x: x.split("context: ")[-1].replace("⋯", "。"), test_gold.question.values))
ture_set = list(map(ans2aop, test_gold.answer.values))

In [ ]:
train = pd.read_json("/workspaces/ABSA/train.json", lines=True)
c_train = train[train["answer"].str.len() > 0]
c_train["context"] = list(map(lambda x: x.split("context: ")[-1].replace("⋯", "。"), c_train.question.values))
train_set = list(map(ans2aop, c_train.answer.values))

In [ ]:
test_gold["context"].shape

In [ ]:
docs = list(nlp.pipe(test_gold["context"].values))
docListEBS = list(map(EBS, docs))
# docListSi = list(map(si, docList))

In [ ]:
ture_set[0]

In [ ]:
docListEBS[0]._.pending_aspects_opinions_text

## CTB9_DEP_ELECTRA_SMALL 

[1~5]

(0.3903985178419504,
 0.4438067643043281,
 **0.5693875870506305,**
 0.47232278939710676)

(0.3416190024775145,
 0.38805714837386507,
 **0.5039559675429242,**
 0.4164505191418483)

[1~6]

(0.3969485422909424,
 0.4424210875380512,
 0.5829314103770624,
 0.4772676213666051)

(0.3477103327699727,
 0.38686595157348763,
 0.5169821925256708,
 0.42118658242431717)

In [ ]:
all_score([i._.pending_aspects_opinions_text for i in docListEBS], list(zip(*ture_set))[0])

In [ ]:
all_score([i._.aspect_sentiment_triplets for i in docListEBS], list(zip(*ture_set))[1])

In [ ]:
docListEBS[0].pos_

In [ ]:
for x, doc in enumerate(zip(docListEBS, ture_set)):
    print(f"---idx[{x}]: {doc[0].text} ---")
    # print(f"---idx[{i}]---")
    # print("POS:", doc[0])
    print("Pred:", doc[0]._.aspect_sentiment_triplets)
    print("Gold:", doc[1][1])

In [ ]:
# t_set = set(list(zip(*ture_set))[0][0])
# p_set = set(docListEBS[0]._.pending_aspects_opinions_text)

In [ ]:
# t_set
# t_set - p_set

In [ ]:
pde = PatternDiscoverEngine()

In [ ]:
# ture_set[78][0]

In [ ]:
# pde.pat_serach(docListEBS[78], ture_set[78][0])

In [ ]:
import pickle

In [ ]:
with open("./docListEBS.pkl", 'wb') as f:
    pickle.dump(docListEBS, f)

In [ ]:
with open("./docListEBS.pkl", "rb") as f:
    docListEBS = pickle.load(f)


In [ ]:
docListEBS

In [ ]:
train_set

In [ ]:
err = []
for i, x in enumerate(zip(docListEBS, train_set)):
    # print(f"---idx[{i}]: {x[0].text} ---")
    try:
        pde.pat_serach(x[0], x[1][0])
    except:
        err.append(i)

In [ ]:
docListEBS[0]._.aspect_sentiment_triplets

In [ ]:
pat_set = pde.get_patterns()

In [ ]:
pat_set.most_common(10)

In [ ]:
pat_set

In [ ]:
ture_set[0][0]

In [ ]:
docListEBS[0][0].head

In [ ]:
# displacy.render(docListEBS[0])

In [ ]:
# displacy.render(docListEBS[267])

In [ ]:
# displacy.render(docListEBS[4])

In [ ]:
# displacy.render(docListEBS[15])

In [ ]:
docListEBS[14]._.aspect_sentiment_triplets

In [ ]:
# displacy.render(docListEBS[9])